# Рекомендация тарифов

## Обзор данных

In [1]:
import pandas as pd #импорт библиотеки pandas

from sklearn.model_selection import train_test_split #импорт функции train_test_split из библиотеки sklearn
#импорт функций accuracy_score, precision_score и recall_score из библиотеки sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score 
from sklearn.tree import DecisionTreeClassifier #импорт модели DecisionTreeClassifier из библиотеки sklearn
from sklearn.ensemble import RandomForestClassifier #импорт модели RandomForestClassifier из библиотеки sklearn
from sklearn.linear_model import LogisticRegression #импорт модели LogisticRegression из библиотеки sklearn

In [2]:
#чтение файла `users_behavior.csv` и сохранение в переменной `df`
df = pd.read_csv('users_behavior.csv')

In [3]:
df.sample(10) # просмотр данных датасета `df`

,calls,minutes,messages,mb_used,is_ultra
3120,93.0,727.11,11.0,17723.74,0
609,51.0,368.20,30.0,20699.67,0
739,36.0,314.67,16.0,16476.83,0
1686,102.0,714.82,34.0,18341.56,0
1813,25.0,180.63,20.0,6854.94,0
3188,78.0,514.37,42.0,16543.09,1
3183,74.0,540.66,18.0,12415.10,0
44,98.0,665.56,106.0,19538.83,0
3052,48.0,359.71,127.0,13830.49,0
2110,44.0,355.66,33.0,17906.06,0


In [4]:
df.info() #получение общей информации о таблице `df`

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Итак, в таблице 5 столбцов. Типы данных в столбцах: float64 и int64.

Согласно документации к данным:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Названия колонок указаны в соответствие с хорошим стилем написания. Количество значений в столбцах одинаковое, значит, в данных нет пропущенных значений. Предобработка данных уже выполнена.

### Выводы

В каждой строке таблицы - данные о поведении одного пользователя за месяц. Часть колонок рассказывает о характеристиках расходуемого трафика: количество звонков, объём минут, количество отправленных сообщений, объём интернет- трафика. Другая часть - о действующем тарифе пользователя: «Смарт» или «Ультра».

Предварительно можно утверждать, что, данных достаточно для построения модели для классификации пользователей с целью их перевода с архивных тарифов на тарифы «Смарт» или «Ультра».

Предобработка данных уже выполнена.

## Подготовка выборок для обучения и тестирования моделей

Так как спрятанной тестовой выборки у нас нет, данные необходимо разбить на три части: обучающую, валидационную и тестовую выборки. Разделим исходный датасет в соотношении 3:1:1 - 60% обучающая выборка и по 20% валидационная и тестовая выборки. Используем для этого функцию train_test_split.

In [5]:
# разделим сначала датасет на обучающую выборку и всё остальное, а затем вторую выборку на валидационною и тестовую
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)

In [6]:
# проверим размеры получившихся выборок
print(df_train.shape)
print(df_valid_test.shape)

(1928, 5)
(1286, 5)


In [7]:
# теперь разделим выборку df_valid_test на валидационною и тестовую
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

In [8]:
# проверим размеры получившихся выборок
print(df_valid.shape)
print(df_test.shape)

(643, 5)
(643, 5)


Также сразу на основе сформированных выборок подготовим отдельные таблицы с признаками, определяющими целевой признак, и целевым признаком.

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_valid.drop(['is_ultra'], axis=1)
target_test = df_valid['is_ultra']

### Выводы

На данном этапе мы разделили исходные данные на обучающую, валидационную и тестовую выборки, а также подготовили таблицы с признаками, определяющими целевой признак, и целевым признаком.

## Исследование моделей

Наш целевой признак (тариф, который будет предлагаться клиентам «Мегалайн» для перехода с архивного тарифа) является категориальным, поэтому мы будем подбирать модель для решения задачи классификации. Так как в нашем случае признака всего два (тарифы «Смарт» или «Ультра»), то нас интересует бинарная классификация.

### Решающее дерево

Используем модель Решающее дерево, попробуем сразу несколько значений гиперпараметра `max_depth` и рассчитаем `accuracy`. Выберем значение `max_depth` при котором получается наилучшее значение `accuracy`.

In [10]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучим модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result # сохраним наилучшее значение метрики accuracy на валидационных данных
        best_depth = depth # сохраним значение depth
        
print("Accuracy лучшей модели:", best_result, "Глубина дерева:", best_depth)

Accuracy лучшей модели: 0.7853810264385692 Глубина дерева: 3


**Вывод**

При использовании модели Решающее дерево наилучшее значение `accuracy` получается 0.79 при значении гиперпараметра Глубина дерева = 3.

### Случайный лес

Теперь используем модель Случайный лес, попробуем сразу несколько значений гиперпараметра `n_estimators` и рассчитаем `accuracy`. Выберем значение `n_estimators` при котором получается наилучшее значение `accuracy`.

In [11]:
best_model = None
best_result = 0
best_estimators = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result # сохраним наилучшее значение метрики accuracy на валидационных данных
        best_estimators = est # сохраним значение n_estimators

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев в лесу:", best_estimators)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 Количество деревьев в лесу: 10


**Вывод**

При использовании модели Случайный лес наилучшее значение `accuracy` получается 0.79 при значении гиперпараметра Количество деревьев в лесу = 10.

### Логистическая регрессия

Используем модель Логистическая регрессия, попробуем сразу несколько значений гиперпараметра `max_iter` и рассчитаем `accuracy`. Выберем значение `max_iter` при котором получается наилучшее значение `accuracy`. Используем алгоритм 'lbfgs'.

In [12]:
best_model = None
best_result = 0
best_iter = 0
for m_iter in range(100, 1001, 100):
    # обучим модель с заданным количеством деревьев
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=m_iter) 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result # сохраним наилучшее значение метрики accuracy на валидационных данных
        best_iter = m_iter # сохраним значение n_estimators

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 
      "Максимальное количество итераций обучения:", best_iter)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715 Максимальное количество итераций обучения: 100


**Вывод**

При использовании модели Логистическая регрессия наилучшее значение `accuracy` получается 0.71 при значении гиперпараметра Максимальное количество итераций обучения = 100.

In [13]:
# составим таблицу сравнения моделей
final = pd.DataFrame({'name': ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'], 
                      'accuracy': [0.79, 0.79, 0.71],
                     'speed': ['Высокая', 'Низкая', 'Высокая']})
final

,name,accuracy,speed
0,Решающее дерево,0.79,Высокая
1,Случайный лес,0.79,Низкая
2,Логистическая регрессия,0.71,Высокая


### Выводы

На данном этапе мы исследовали качество разных моделей, меняя их гиперпараметры. Исходя из полученных результатов наилучшей является модель Решающее дерево при значении гиперпараметра Глубина дерева = 3, однако перед принятием окончательного решения о выборе модели требуется проверка моделей на тестовой выборке.

## Проверка модели на тестовой выборке

In [14]:
# проверим модель Решающее дерево на тестовой выборке
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.7853810264385692

In [15]:
# проверим модель Случайный лес на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.7853810264385692

In [16]:
# проверим модель Логистическая регрессия на тестовой выборке
model = LogisticRegression(random_state=12345, solver='lbfgs') 
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
result

0.7107309486780715

### Выводы

Значения `accuracy` на тестовой выборке получились такими же как и на валидационной, соответственно вывод не меняется - наилучшей является модель Решающее дерево при значении параметра Глубина дерева = 3.

## Проверка модели на адекватность

Для проверки модели на адекватность посчитаем дополнительные метрики precision и recall.

In [17]:
# посчитаем precision для модели Решающее дерево на тестовой выборке
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
precision_score(target_test, predictions_test)

0.7217391304347827

In [18]:
# посчитаем precision для модели Случайный лес на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
precision_score(target_test, predictions_test)

0.6946564885496184

In [19]:
# посчитаем precision для модели Логистическая регрессия на тестовой выборке
model = LogisticRegression(random_state=12345, solver='lbfgs') 
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
precision_score(target_test, predictions_test)

0.5882352941176471

In [20]:
# посчитаем recall для модели Решающее дерево на тестовой выборке
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
recall_score(target_test, predictions_test)

0.43915343915343913

In [21]:
# посчитаем recall для модели Случайный лес на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
recall_score(target_test, predictions_test)

0.48148148148148145

In [22]:
# посчитаем recall для модели Логистическая регрессия на тестовой выборке
model = LogisticRegression(random_state=12345, solver='lbfgs') 
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
recall_score(target_test, predictions_test)

0.05291005291005291

In [23]:
# соберём значения всех метрик в общую таблицу
final['precision'] = ([0.72, 0.69, 0.58])
final['recall'] = ([0.44, 0.48, 0.05])
final

,name,accuracy,speed,precision,recall
0,Решающее дерево,0.79,Высокая,0.72,0.44
1,Случайный лес,0.79,Низкая,0.69,0.48
2,Логистическая регрессия,0.71,Высокая,0.58,0.05


In [24]:
# изменим порядок столбцов для удобства чтения
final = final[['name', 'accuracy', 'precision', 'recall', 'speed']]
final

,name,accuracy,precision,recall,speed
0,Решающее дерево,0.79,0.72,0.44,Высокая
1,Случайный лес,0.79,0.69,0.48,Низкая
2,Логистическая регрессия,0.71,0.58,0.05,Высокая


### Выводы

По итогам сравнения значений метрик `accuracy`, `precision`, `recall` и с учётом скорости работы модели выбираем модель Решающее дерево.

## Общий вывод

Исследование проводилось в пять этапов:
* На этапе Обзор данных мы ознакомились с данными в предоставленной таблице, убедились, что предобработка данных выполнена, и зафиксировали, что предварительно данных для проведения исследования достаточно;
* На этапе Подготовка выборок для обучения и тестирования моделей мы разделили исходный датасет на 3 выборки: обучающую, валидационную и тестовую выборки, а также подготовили таблицы с признаками, определяющими целевой признак, и целевым признаком;
* На этапе Исследование моделей мы исследовали качество моделей Решающее дерево, Случайный лес и Логистическая регрессия с разными значениями их гиперпараметров и определили, что наилучшей является модель Решающее дерево при значении гиперпараметра Глубина дерева = 3;
* На этапе  Проверка модели на тестовой выборке рассчитали метрику `accuracy` на каждой модели для тестовой выборки и получили те же значения, что и для валидационной выборки;
* На этапе Проверка модели на адекватность рассчитали ещё дополнительные метрики `precision`, `recall` и свели все значения в общую таблицу.

По итогам всех проделанных этапов наилучшей моделью для задачи классификации поведения клиентов на архивных тарифах и подбора им подходящего нового тарифа: «Смарт» или «Ультра» - является иодель Решающее дерево при значении гиперпараметра Глубина дерева = 3. В этом случае значением `accuracy` = 0.79. Для модели Случайный лес также получается достичь такого значения `accuracy`, но выбираем модель Решающее дерево по критерию скорости работы.